In [1]:
#Project Structure
image_classification/
│
├── app/
│   ├── __init__.py
│   ├── routes.py
│   ├── static/
│       └── uploads/
│   └── templates/
│       ├── index.html
│       └── result.html
├── model/
│   ├── train_model.py
│   └── cnn_model.h5
├── data/
├── app.py
└── requirements.txt


SyntaxError: invalid character '│' (U+2502) (<ipython-input-1-3253a787622e>, line 3)

In [ ]:
# Dependencies (requirements.txt)
Flask==2.1.1
numpy==1.23.1
tensorflow==2.9.1
keras==2.9.0
pillow==9.2.0
matplotlib==3.5.2



In [ ]:
#Model Training (model/train_model.py)
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Load and preprocess the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

# Save the model
model.save('../model/cnn_model.h5')

# Plot training and validation accuracy over epochs
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(f'Test accuracy: {test_acc}')



In [2]:
#Flask Application (app/routes.py)
from flask import Flask, render_template, request, redirect, url_for
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

app = Flask(__name__)
model = load_model('model/cnn_model.h5')

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return redirect(url_for('index'))

    file = request.files['file']
    if file.filename == '':
        return redirect(url_for('index'))

    filepath = os.path.join('app/static/uploads', file.filename)
    file.save(filepath)

    img = load_img(filepath, target_size=(32, 32))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    predicted_class = classes[np.argmax(predictions)]

    return render_template('result.html', image_url=filepath, prediction=predicted_class)

if __name__ == '__main__':
    app.run(debug=True)



FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'model/cnn_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
#app/templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Image Classification</title>
</head>
<body>
    <h1>Image Classification with CNN</h1>
    <form action="/predict" method="post" enctype="multipart/form-data">
        <input type="file" name="file"><br><br>
        <input type="submit" value="Upload and Classify">
    </form>
</body>
</html>


In [ ]:
#app/templates/result.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Classification Result</title>
</head>
<body>
    <h1>Classification Result</h1>
    <img src="{{ url_for('static', filename='uploads/' + image_url) }}" alt="Uploaded Image" width="200"><br><br>
    <p>Prediction: {{ prediction }}</p>
    <a href="/">Classify another image</a>
</body>
</html>



In [ ]:
#Flask Application Entry Point (app.py)
from app import app

if __name__ == '__main__':
    app.run(debug=True)
